In [1]:
#Load Basic libraries
import numpy as np

#Data
import os
import csv

#Model
import keras
from keras.utils import to_categorical
from keras.utils import get_file
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten, Activation, Dropout

from keras.models import model_from_json

print("All libraries loaded!")


Using TensorFlow backend.
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

All libraries loaded!


In [2]:
#Función auxiliar que redimensiona un array dado
def flatten(dimData, moves):
    moves = np.array(moves)
    moves = moves.reshape(len(moves), dimData)
    moves = moves.astype('float32')
    moves /= 255
    return moves


In [3]:
#Carga los datos
os.chdir("D:/Documentos/UCM/TFG/Red Neuronal")
os.getcwd()


moves, labels = [],[]
dirMoves = ["UP.csv", "DOWN.csv", "RIGHT.csv", "LEFT.csv"]

#Guarda todos los archivos de datos
for dir in dirMoves:
    #Este encoding es para que no genere caracteres extraños
    file = open(dirMoves[dirMoves.index(dir)], encoding='utf-8-sig')
    reader = csv.reader(file)

    for row in reader:
        #print(row)
        moves.append(row)
        labels.append(dirMoves.index(dir))

In [4]:
#Separa el conjunto de datos en datos de prueba y de entrenamiento
#Se necesitaran mas de entrenamiento que de prueba
segregation, index = 5,0
train_moves, test_moves, train_labels, test_labels = [],[],[],[]

for move, label in zip(moves, labels):
    if index < segregation:
        train_moves.append(move)
        train_labels.append(label)
        index += 1
    else:
        test_moves.append(move)
        test_labels.append(label)
        index = 0

print('Numero de movimientos de entrenamiento: ', len(train_moves))
print('Numero de movimientos de prueba: ', len(test_moves))

Numero de movimientos de entrenamiento:  481
Numero de movimientos de prueba:  96


In [5]:
#"Aplana" los datos con la funcion auxiliar
dataDim = np.prod(len(moves[0]))
train_data  = flatten(dataDim, train_moves)
test_data = flatten(dataDim, test_moves)


train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

#Determina el número de clases que se van a utilizar 
classes = np.unique(train_labels)
nClasses  = len(classes)

In [6]:
#Para esta red neuronal se establecen tres capas, y 256 neuronas
model = Sequential()
model.add(Dense(256, activation = 'tanh', input_shape = (dataDim,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))


epochs = 100;
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_one_hot, batch_size = 256, epochs=epochs, verbose=1,
                    validation_data=(test_data, test_labels_one_hot))

#test model
[test_loss, test_acc] = model.evaluate(test_data, test_labels_one_hot)
print("Resultado de la evaluación : Perdidas = {}, Precisión = {}".format(test_loss, test_acc))

model.summary()


Train on 481 samples, validate on 96 samples
Epoch 1/100
481/481 [==============================] - 3s 5ms/step - loss: 1.3798 - accuracy: 0.3077 - val_loss: 1.3515 - val_accuracy: 0.4583
Epoch 2/100
481/481 [==============================] - 0s 52us/step - loss: 1.3453 - accuracy: 0.3992 - val_loss: 1.3070 - val_accuracy: 0.3958
Epoch 3/100
481/481 [==============================] - 0s 33us/step - loss: 1.2902 - accuracy: 0.4699 - val_loss: 1.2334 - val_accuracy: 0.5521
Epoch 4/100
481/481 [==============================] - 0s 35us/step - loss: 1.2216 - accuracy: 0.6029 - val_loss: 1.1593 - val_accuracy: 0.6667
Epoch 5/100
481/481 [==============================] - 0s 33us/step - loss: 1.1371 - accuracy: 0.6861 - val_loss: 1.0558 - val_accuracy: 0.7083
Epoch 6/100
481/481 [==============================] - 0s 33us/step - loss: 1.0616 - accuracy: 0.6736 - val_loss: 0.9833 - val_accuracy: 0.6979
Epoch 7/100
481/481 [==============================] - 0s 34us/step - loss: 0.9756 - accura

In [7]:
#Serializa el modelo para JSON
model_json = model.to_json()
model_path = "D:/Documentos/UCM/TFG/Red Neuronal/Modelo/model.json"
with open(model_path, "w") as json_file:
  json_file.write(model_json)

weights_path = "D:/Documentos/UCM/TFG/Red Neuronal/Modelo/model.h5"
#Serializa los pesos (weights) para HDF5
model.save_weights(weights_path)

#Carga el json y crea el modelo
json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
 
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#Carga los pesos (weights) en el nuevo modelo
loaded_model.load_weights(weights_path)

#Test
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               38656     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                

In [18]:
import keras2onnx as k2o
import onnx
import onnxruntime

# convert to onnx model
onnx_model = k2o.convert_keras(model, model.name)

#Save de onnx model
temp_model_file = "D:/Documentos/UCM/TFG/Red Neuronal/Modelo/ONNX/model.onnx"
k2o.save_model(onnx_model, temp_model_file)

try:
    sess = onnxruntime.InferenceSession(temp_model_file)
    ok = True    
except (InvalidGraph, TypeError, RuntimeError) as e:
    # Probably a mismatch between onnxruntime and onnx version.
    print(e)
    ok = False

if ok:
    print("The model expects input shape:", sess.get_inputs()[0].shape)
    print("x shape:", train_data.shape)
    

    x = train_data if isinstance(train_data, list) else [train_data]
        
    feed = dict([(input.name, x[n]) for n, input in enumerate(sess.get_inputs())])
    pred_onnx = sess.run(None, feed)

    prob = pred_onnx[0]
    print(prob.ravel()[:10])
    

The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 9.


The model expects input shape: ['N', 150]
x shape: (481, 150)
[7.5305969e-01 9.4845602e-03 2.3739211e-01 6.3674073e-05 9.9798536e-01
 1.5387441e-04 9.4447525e-05 1.7663178e-03 9.9778819e-01 1.1104543e-03]
